# Dependencies 

In [1]:
import os
import HPCP
import json
import numpy as np
import pandas
import keras
import math
import regul
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
import time
from tensorflow import keras
from tensorflow.keras import layers
import pyaudio
import wave

# Converting .wav to .numpy format

In [2]:
chords_dir = 'Guitar_Only'
if os.path.isdir(chords_dir):
    chords = []
    for c in os.listdir(chords_dir):
        if not os.path.isfile(c):
            chords.append(c)
    chords.sort()
    #print(chords)
else:
    chords = ['a', 'am', 'bm', 'c', 'd', 'dm', 'e', 'em', 'f', 'g']
semi_tones = ['c', 'c#', 'd', 'd#', 'e', 'f', 'f#', 'g', 'g#', 'a', 'a#', 'b']
total_chords = []
for c in chords:
    total_chords.append(len(os.listdir(chords_dir + '/'+ c)))
#print(total_chords)

In [3]:
def generate(chords_dir,chords,semi_tones,total_chords):
    chromas = np.zeros((sum(total_chords),13))
    k=0
    for i in range(len(chords)):
        for j in range(total_chords[i]):
            file = chords[i] + str(j+1) + '.wav'
            path = chords_dir + '/' + chords[i] + '/' + file
            chroma = HPCP.hpcp(path, norm_frames=False, win_size=4096, hop_size=1024, output='numpy')
            chroma = np.mean(chroma, axis=0)
            chroma /= sum(chroma)
            chroma = np.append(chroma, i)
            chromas[k]=chroma
            k+=1
    np.savetxt('pcp.data', chromas, delimiter=',')

In [36]:
generate(chords_dir,chords,semi_tones,total_chords)

# Data split into train and test set

In [4]:
data = np.loadtxt('pcp.data', delimiter=',') # load the data
X = data[:,:12]
Y = data[:,12]
m=data.shape[0]
Y = Y.astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
y_train = y_train.reshape((-1,1))
y_test = y_test.reshape((-1,1))

# Model

In [6]:
checkpoint_path = "wts/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
inputs = keras.Input(shape=(12,))
dense = layers.Dense(1000, activation="relu")
x = dense(inputs)
x = layers.Dropout(0.3)(x)
x = layers.Dense(35, activation="relu")(x)
outputs = layers.Dense(len(chords))(x)
model = keras.Model(inputs=inputs, outputs=outputs)
#model.summary()

In [7]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.SGD(learning_rate=0.01, momentum=0.95),
    metrics=["accuracy"],)

In [10]:
# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,save_weights_only=True,verbose=1)
history = model.fit(X_train, y_train, batch_size=300, epochs=10000, validation_split=0.2,callbacks=[cp_callback])

Epoch 1/10000
1/4 [======>.......................] - ETA: 0s - loss: 0.0101 - accuracy: 0.9967
Epoch 00001: saving model to wts/cp.ckpt
4/4 [==============================] - 0s 28ms/step - loss: 0.0088 - accuracy: 0.9981 - val_loss: 0.6352 - val_accuracy: 0.9254
Epoch 2/10000
1/4 [======>.......................] - ETA: 0s - loss: 0.0091 - accuracy: 1.0000
Epoch 00002: saving model to wts/cp.ckpt
4/4 [==============================] - 0s 12ms/step - loss: 0.0073 - accuracy: 0.9991 - val_loss: 0.6362 - val_accuracy: 0.9254
Epoch 3/10000
1/4 [======>.......................] - ETA: 0s - loss: 0.0052 - accuracy: 1.0000
Epoch 00003: saving model to wts/cp.ckpt
4/4 [==============================] - 0s 12ms/step - loss: 0.0078 - accuracy: 0.9991 - val_loss: 0.6276 - val_accuracy: 0.9254
Epoch 4/10000
1/4 [======>.......................] - ETA: 0s - loss: 0.0055 - accuracy: 1.0000
Epoch 00004: saving model to wts/cp.ckpt
4/4 [==============================] - 0s 12ms/step - loss: 0.0090 - acc

Epoch 32/10000
1/4 [======>.......................] - ETA: 0s - loss: 0.0102 - accuracy: 1.0000
Epoch 00032: saving model to wts/cp.ckpt
4/4 [==============================] - 0s 10ms/step - loss: 0.0089 - accuracy: 0.9991 - val_loss: 0.6317 - val_accuracy: 0.9216
Epoch 33/10000
1/4 [======>.......................] - ETA: 0s - loss: 0.0155 - accuracy: 0.9967
Epoch 00033: saving model to wts/cp.ckpt
4/4 [==============================] - 0s 10ms/step - loss: 0.0096 - accuracy: 0.9972 - val_loss: 0.6160 - val_accuracy: 0.9179
Epoch 34/10000
1/4 [======>.......................] - ETA: 0s - loss: 0.0076 - accuracy: 0.9967
Epoch 00034: saving model to wts/cp.ckpt
4/4 [==============================] - 0s 12ms/step - loss: 0.0075 - accuracy: 0.9981 - val_loss: 0.6149 - val_accuracy: 0.9179
Epoch 35/10000
1/4 [======>.......................] - ETA: 0s - loss: 0.0050 - accuracy: 1.0000
Epoch 00035: saving model to wts/cp.ckpt
4/4 [==============================] - 0s 11ms/step - loss: 0.0057 -

Epoch 63/10000
1/4 [======>.......................] - ETA: 0s - loss: 0.0083 - accuracy: 0.9967
Epoch 00063: saving model to wts/cp.ckpt
4/4 [==============================] - 0s 10ms/step - loss: 0.0128 - accuracy: 0.9963 - val_loss: 0.6175 - val_accuracy: 0.9216
Epoch 64/10000
1/4 [======>.......................] - ETA: 0s - loss: 0.0164 - accuracy: 0.9967
Epoch 00064: saving model to wts/cp.ckpt
4/4 [==============================] - 0s 10ms/step - loss: 0.0107 - accuracy: 0.9991 - val_loss: 0.6422 - val_accuracy: 0.9254
Epoch 65/10000
1/4 [======>.......................] - ETA: 0s - loss: 0.0029 - accuracy: 1.0000
Epoch 00065: saving model to wts/cp.ckpt
4/4 [==============================] - 0s 11ms/step - loss: 0.0104 - accuracy: 0.9963 - val_loss: 0.6218 - val_accuracy: 0.9254
Epoch 66/10000
1/4 [======>.......................] - ETA: 0s - loss: 0.0036 - accuracy: 1.0000
Epoch 00066: saving model to wts/cp.ckpt
4/4 [==============================] - 0s 12ms/step - loss: 0.0082 -

Epoch 94/10000
1/4 [======>.......................] - ETA: 0s - loss: 0.0180 - accuracy: 0.9933
Epoch 00094: saving model to wts/cp.ckpt
4/4 [==============================] - 0s 10ms/step - loss: 0.0105 - accuracy: 0.9981 - val_loss: 0.6178 - val_accuracy: 0.9179
Epoch 95/10000
1/4 [======>.......................] - ETA: 0s - loss: 0.0117 - accuracy: 0.9967
Epoch 00095: saving model to wts/cp.ckpt
4/4 [==============================] - 0s 11ms/step - loss: 0.0110 - accuracy: 0.9972 - val_loss: 0.5996 - val_accuracy: 0.9291
Epoch 96/10000
1/4 [======>.......................] - ETA: 0s - loss: 0.0160 - accuracy: 0.9933
Epoch 00096: saving model to wts/cp.ckpt
4/4 [==============================] - 0s 10ms/step - loss: 0.0136 - accuracy: 0.9963 - val_loss: 0.6146 - val_accuracy: 0.9291
Epoch 97/10000
1/4 [======>.......................] - ETA: 0s - loss: 0.0148 - accuracy: 0.9967
Epoch 00097: saving model to wts/cp.ckpt
4/4 [==============================] - 0s 10ms/step - loss: 0.0097 -

Epoch 125/10000
1/4 [======>.......................] - ETA: 0s - loss: 0.0082 - accuracy: 1.0000
Epoch 00125: saving model to wts/cp.ckpt
4/4 [==============================] - 0s 12ms/step - loss: 0.0063 - accuracy: 0.9991 - val_loss: 0.6190 - val_accuracy: 0.9216
Epoch 126/10000
1/4 [======>.......................] - ETA: 0s - loss: 0.0098 - accuracy: 0.9967
Epoch 00126: saving model to wts/cp.ckpt
4/4 [==============================] - 0s 10ms/step - loss: 0.0073 - accuracy: 0.9991 - val_loss: 0.6207 - val_accuracy: 0.9328
Epoch 127/10000
1/4 [======>.......................] - ETA: 0s - loss: 0.0074 - accuracy: 1.0000
Epoch 00127: saving model to wts/cp.ckpt
4/4 [==============================] - 0s 10ms/step - loss: 0.0055 - accuracy: 1.0000 - val_loss: 0.6162 - val_accuracy: 0.9291
Epoch 128/10000
1/4 [======>.......................] - ETA: 0s - loss: 0.0158 - accuracy: 0.9933
Epoch 00128: saving model to wts/cp.ckpt
4/4 [==============================] - 0s 10ms/step - loss: 0.01

Epoch 156/10000
1/4 [======>.......................] - ETA: 0s - loss: 0.0074 - accuracy: 1.0000
Epoch 00156: saving model to wts/cp.ckpt
4/4 [==============================] - 0s 12ms/step - loss: 0.0084 - accuracy: 0.9981 - val_loss: 0.6347 - val_accuracy: 0.9291
Epoch 157/10000
1/4 [======>.......................] - ETA: 0s - loss: 0.0051 - accuracy: 1.0000
Epoch 00157: saving model to wts/cp.ckpt
4/4 [==============================] - 0s 12ms/step - loss: 0.0045 - accuracy: 1.0000 - val_loss: 0.6376 - val_accuracy: 0.9254
Epoch 158/10000
1/4 [======>.......................] - ETA: 0s - loss: 0.0067 - accuracy: 1.0000
Epoch 00158: saving model to wts/cp.ckpt
4/4 [==============================] - 0s 11ms/step - loss: 0.0085 - accuracy: 0.9981 - val_loss: 0.6403 - val_accuracy: 0.9291
Epoch 159/10000
1/4 [======>.......................] - ETA: 0s - loss: 0.0082 - accuracy: 0.9967
Epoch 00159: saving model to wts/cp.ckpt
4/4 [==============================] - 0s 11ms/step - loss: 0.00

Epoch 187/10000
1/4 [======>.......................] - ETA: 0s - loss: 0.0078 - accuracy: 0.9967
Epoch 00187: saving model to wts/cp.ckpt
4/4 [==============================] - 0s 11ms/step - loss: 0.0069 - accuracy: 0.9981 - val_loss: 0.6147 - val_accuracy: 0.9216
Epoch 188/10000
1/4 [======>.......................] - ETA: 0s - loss: 0.0047 - accuracy: 1.0000
Epoch 00188: saving model to wts/cp.ckpt
4/4 [==============================] - 0s 10ms/step - loss: 0.0059 - accuracy: 0.9991 - val_loss: 0.6188 - val_accuracy: 0.9216
Epoch 189/10000
1/4 [======>.......................] - ETA: 0s - loss: 0.0074 - accuracy: 1.0000
Epoch 00189: saving model to wts/cp.ckpt
4/4 [==============================] - 0s 12ms/step - loss: 0.0062 - accuracy: 1.0000 - val_loss: 0.6262 - val_accuracy: 0.9254
Epoch 190/10000
1/4 [======>.......................] - ETA: 0s - loss: 0.0036 - accuracy: 1.0000
Epoch 00190: saving model to wts/cp.ckpt
4/4 [==============================] - 0s 11ms/step - loss: 0.00

KeyboardInterrupt: 

In [11]:
test_scores = model.evaluate(X_test, y_test, verbose=2)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

21/21 - 0s - loss: 0.3281 - accuracy: 0.9348
Test loss: 0.32806795835494995
Test accuracy: 0.9348484873771667


In [12]:
predictions = model.predict(X_test[:2])
print("predictions shape:", predictions)
model.load_weights(checkpoint_path)

predictions shape: [[-33.09671   -29.291353    8.486401   11.325174    2.882259   13.418601
   -0.2805896  17.041925   -7.955277   38.497528 ]
 [ 30.631968   43.560394   -2.585856    6.1264      1.5229123  19.692543
  -10.568228  -54.941967   33.07732   -47.03671  ]]


In [13]:
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
RECORD_SECONDS = 3
WAVE_OUTPUT_FILENAME = "output.wav"
while True:
    p = pyaudio.PyAudio()

    stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

    print("* recording")

    frames = []

    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)

    print("* done recording")

    stream.stop_stream()
    stream.close()
    p.terminate()

    wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()
    chroma = HPCP.hpcp(WAVE_OUTPUT_FILENAME, norm_frames=False, win_size=4096, hop_size=1024, output='numpy')
    chroma = np.mean(chroma, axis=0)
    chroma /= sum(chroma)
    chroma = chroma.reshape((1,12))
    print(chroma.shape)
    predictions = model.predict(chroma)
    print("predictions array:", predictions)
    break

* recording
* done recording
(1, 12)
predictions array: [[-89.56327     2.5124266  35.782433   21.237513   -8.311157   -3.3832831
   21.107727   25.153713    9.060873   14.787994 ]]


In [89]:
pa = pyaudio.PyAudio()
pa.get_default_input_device_info()


{'index': 0,
 'structVersion': 2,
 'name': 'MacBook Pro Microphone',
 'hostApi': 0,
 'maxInputChannels': 1,
 'maxOutputChannels': 0,
 'defaultLowInputLatency': 0.048480725623582764,
 'defaultLowOutputLatency': 0.01,
 'defaultHighInputLatency': 0.05863945578231292,
 'defaultHighOutputLatency': 0.1,
 'defaultSampleRate': 44100.0}